In [3]:
import os
import tkinter as tk
from tkinter import filedialog
from RelateMismatchesToFeature import relateMismatchesToFeature
tk.Tk().withdraw()
bioinformaticsDir = filedialog.askdirectory(title = "Choose Bioinformatics Directory")
deaminationDeterminationDataDir = os.path.join(bioinformaticsDir, "deamination_determination", "data")
TFBS_Dir = os.path.join(deaminationDeterminationDataDir, "TFBS")
TFBS_MidpointFilePaths = {"CTCF_known1_DHS":os.path.join(TFBS_Dir, "CTCF_known1_DHS_midpoint.bed"),
                          "CTCF_known1_non-DHS":os.path.join(TFBS_Dir, "CTCF_known1_non-DHS_midpoint.bed")}
mutperiodExternalDataDir = os.path.join(bioinformaticsDir,"mutperiod","mutperiod_data","__external_data")
nucleosomeFilePaths = {"hg19_mnase_all":os.path.join(mutperiodExternalDataDir,"hg19", "hg19_MNase_nucleosome_map_all","hg19_MNase_nucleosome_map_all.bed"),
                       "hg19_LCL_mnase_all":os.path.join(mutperiodExternalDataDir,"hg19", "hg19_LCL_MNase_nuc_map_all","hg19_LCL_MNase_nuc_map_all.bed"),
                       "hg19_LCL_mnase_all_mappable":os.path.join(mutperiodExternalDataDir,"hg19", "hg19_LCL_MNase_nuc_map_all_mappable","hg19_LCL_MNase_nuc_map_all_mappable.bed")}
# nucleosomeFilePaths = {"hg19_LCL_mnase_all_mappable_100bp":os.path.join(mutperiodExternalDataDir, "hg19",
#                                                                         "hg19_LCL_MNase_nuc_map_all_mappable_100bp_radius", "hg19_LCL_MNase_nuc_map_all_mappable_100bp_radius.bed")}
mismatchesByReadDirectories = dict()
cellTypes = ("NHF1",)
timepointsByCellType = {"NHF1":["1h", "8h", "24h", "48h"]}
for cellType in cellTypes:
    mismatchesByReadDirectories[cellType] = os.path.join(deaminationDeterminationDataDir, cellType, "mismatches_by_read")

In [4]:
for cellType in cellTypes:
    for timepoint in timepointsByCellType[cellType]:
        for mismatchType in ("C_to_T", "CC_to_TT", "C_to_T_CC_to_TT"):
            
            print(f"\nWorking with: {cellType}; {timepoint}; {mismatchType}")

            formattedMismatchesByReadFilePath = os.path.join(mismatchesByReadDirectories[cellType],
                                                             f"{cellType}_CPD_{timepoint}_all_reps_{mismatchType}_mismatches_by_read_relation_formatted_TGG_filtered.bed")

            print("\tRelating TFBSs...")
            for TFBS_Midpoint in TFBS_MidpointFilePaths:
                print(f"\tWorking with {TFBS_Midpoint}")
                relateMismatchesToFeature(formattedMismatchesByReadFilePath, TFBS_MidpointFilePaths[TFBS_Midpoint],
                                        outputSuffix = f"_{TFBS_Midpoint}_related", enforcedStrand = '+')
            
            print("\tRelating nucleosomes...")
            for nucleosomeMap in nucleosomeFilePaths:
                print(f"\tWorking with {nucleosomeMap}")
                relateMismatchesToFeature(formattedMismatchesByReadFilePath, nucleosomeFilePaths[nucleosomeMap],
                                        outputSuffix = f"_{nucleosomeMap}_related", enforcedStrand = '+', midpointRadius = 500)


Working with: NHF1; 1h; C_to_T
	Relating TFBSs...
	Working with CTCF_known1_DHS
	Working with CTCF_known1_non-DHS
	Relating nucleosomes...
	Working with hg19_LCL_mnase_all_mappable_100bp

Working with: NHF1; 1h; CC_to_TT
	Relating TFBSs...
	Working with CTCF_known1_DHS
	Working with CTCF_known1_non-DHS
	Relating nucleosomes...
	Working with hg19_LCL_mnase_all_mappable_100bp

Working with: NHF1; 1h; C_to_T_CC_to_TT
	Relating TFBSs...
	Working with CTCF_known1_DHS
	Working with CTCF_known1_non-DHS
	Relating nucleosomes...
	Working with hg19_LCL_mnase_all_mappable_100bp

Working with: NHF1; 8h; C_to_T
	Relating TFBSs...
	Working with CTCF_known1_DHS
	Working with CTCF_known1_non-DHS
	Relating nucleosomes...
	Working with hg19_LCL_mnase_all_mappable_100bp

Working with: NHF1; 8h; CC_to_TT
	Relating TFBSs...
	Working with CTCF_known1_DHS
	Working with CTCF_known1_non-DHS
	Relating nucleosomes...
	Working with hg19_LCL_mnase_all_mappable_100bp

Working with: NHF1; 8h; C_to_T_CC_to_TT
	Relat